# GOALS FOR TOMORROW:

* Build some crappy models

In [1]:
## testing printing output from console
import subprocess
cmd = [ 'echo', '"Welcome to my PySpark analysis of some StackExchange Data"' ]
output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
print(output)

b'"Welcome to my PySpark analysis of some StackExchange Data"\n'


In [2]:
## import standard libraries
import os
import re
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

#%matplotlib inline

# Load PySpark

In [3]:
%run -i '1-load-pyspark.py'

The Spark UI is available at: http://192.168.0.26:4040/ and the defaultParallelism is 4


# Load Data

In [4]:
%%time
print(datetime.now().time())
%run -i '2-load-datasets.py'

12:14:28.164011
------------------------
english
------------------------
root
 |-- _Body: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)
 |-- _Score: long (nullable = true)

+--------------------+--------------------+----------+------+
|               _Body|              _Title|_ViewCount|_Score|
+--------------------+--------------------+----------+------+
|<p>How do I know ...|What is the diffe...|     18413|    35|
|<p>When you want ...|Should I use a se...|    106724|    52|
|<blockquote>
  <p...|What does Maugham...|      1131|    11|
+--------------------+--------------------+----------+------+
only showing top 3 rows

------------------------
math
------------------------
root
 |-- Body: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- __index_level_0__: long (nullable = true)

+--------------------+--------------------+--------

In [5]:
def show_spark_df(df, n=5):
    '''
    function to better print spark df entries
    '''
    display(pd.DataFrame(df.head(n), columns=df.columns))
    
show_spark_df(datasets["english"])

,_Body,_Title,_ViewCount,_Score
0,<p>How do I know when to use <em>lay</em> and ...,"What is the difference between ""lay"" and ""lie""?",18413,35
1,<p>When you want to connect two closely relate...,Should I use a semicolon or a dash to connect ...,106724,52
2,<blockquote>\n <p><strong>Possible Duplicate:...,"What does Maugham mean by ""his spaghetti were""?",1131,11
3,<p>How do you say it correctly?</p>\n\n<ul>\n<...,"""Adult and children stories"" or ""Adults and ch...",959,2
4,"<p>""Proven"" and ""proved"" both seem to mean the...","What is the difference between ""proven"" and ""p...",52711,50


# Clean Data

In [6]:
%%time
print(datetime.now().time())
%run -i '3-clean-datasets.py'

12:14:36.000223

 checking columns are the right types and names 

----- english -----
root
 |-- body: string (nullable = true)
 |-- title: string (nullable = true)
 |-- viewcount: long (nullable = true)
 |-- score: long (nullable = true)
 |-- clean_body: string (nullable = true)

None
----- math -----
root
 |-- body: string (nullable = true)
 |-- title: string (nullable = true)
 |-- viewcount: long (nullable = true)
 |-- score: long (nullable = true)
 |-- clean_body: string (nullable = true)

None
----- rus_stackoverflow -----
root
 |-- body: string (nullable = true)
 |-- title: string (nullable = true)
 |-- viewcount: long (nullable = true)
 |-- score: long (nullable = true)
 |-- clean_body: string (nullable = true)

None
----- stackoverflow -----
root
 |-- body: string (nullable = true)
 |-- title: string (nullable = true)
 |-- viewcount: long (nullable = true)
 |-- score: long (nullable = true)
 |-- clean_body: string (nullable = true)

None
----- superuser -----
root
 |-- body: st

# EDA (optional)

%%time
print(datetime.now().time())
%run -i '4-eda.py'

'''NB TO DO: Find threshold to delete low views to make sure users that can vote have seen the question'''

vc_thresh_data = {}

## finding means of viewcounts across fora

for i in data_array:
    vc_thresh_data[i] = datasets[i].select("viewcount").rdd.flatMap(lambda x: x).mean()

vc_thresh_data

# Define Ravi Target Variable

In [7]:
%%time
print(datetime.now().time())
%run -i '5-define-target.py'

12:15:35.295264

The average value of english y_ravi is 0.0037778


The average value of math y_ravi is 0.0121854


The average value of rus_stackoverflow y_ravi is 0.006857


The average value of stackoverflow y_ravi is 0.002148


The average value of superuser y_ravi is 0.0027184

CPU times: user 243 ms, sys: 68.5 ms, total: 311 ms
Wall time: 56 s


# Validation

%%time

## good/bad threshold validation

## choose thresholds
threshes = np.linspace(0.05, 0.95, num=37)

## create empty list for results
result = []

## create user-defined function to add threshold columns
def thresh_cols(a, high_threshold, low_threshold):
    '''function to add threshold columns'''        
    if a > high_threshold:
        return 'good'
    elif a <= low_threshold:
        return 'bad'

thresh_cols_udf = udf(thresh_cols, StringType())
   
for i in range(len(threshes)):
    ## store thresholds, lowest represents % good questions
    lo_thresh = np.quantile(datasets['english'].select('y_ravi').
                            rdd.flatMap(lambda x: x).collect(), threshes[i])
    hi_thresh = np.quantile(datasets['english'].select('y_ravi').
                            rdd.flatMap(lambda x: x).collect(), threshes[len(threshes)-i-1])
    
    # if statement to change over at 50% threshold
    if (i > (len(threshes) - 1) / 2):
        ## label questions according to high and low thresholds AFTER 50% THRESHOLD
        # here we only need one threshold
        datasets['english'] = datasets['english'].withColumn("col." + str(round(threshes[i], 3)),
                                                             thresh_cols_udf('y_ravi', lit(hi_thresh), lit(hi_thresh)))
    else:
        ## label questions according to high and low thresholds BEFORE 50% THRESHOLD
        # here there are two mirrored threshold, where NAs fill the middle
        datasets['english'] = datasets['english'].withColumn("col." + str(round(threshes[i], 3)),
                                                             thresh_cols_udf('y_ravi', lit(hi_thresh), lit(lo_thresh)))

'''
## get tokens and remove whitespace that messes up French and Linguistics
    toks_one <- tokens_remove(tokens(paste0(
      sxdf_q[sxdf_q[[as.character(threshes[i])]]=="good",][["Body"[1]]], collapse = " ")), 
                              "\\p{Z}", valuetype = "regex")
    toks_two <- tokens_remove(tokens(paste0(
      sxdf_q[sxdf_q[[as.character(threshes[i])]]=="bad",][["Body"[1]]], collapse = " ")), 
                              "\\p{Z}", valuetype = "regex")
    ## reassign docnames
    docnames(toks_one) <- "good"
    docnames(toks_two) <- "bad"
    
    ## we normalise the dfm by stemming, removing stopwords/punctuation and weighting proportionally
    docfm <- dfm_weight(dfm(c(toks_one, toks_two),
                            stem = TRUE, remove_punct = TRUE, remove = stopwords("english")),
                        "prop")
    
    ## bind results
    result <- rbind(result, c(threshes[i], 
                              textstat_dist(docfm, method="euclidean"), 
                              textstat_simil(docfm, method="cosine"), 
                              textstat_simil(docfm, method="jaccard"), 
                              datasets[k]))
'''

# Pipeline

In [8]:
## garbage collector to speed up computation
import gc
collected = gc.collect()
print("Garbage collector: collected %d objects." % collected)

Garbage collector: collected 178 objects.


In [9]:
from pyspark.ml.pipeline import Pipeline
'''MUST STILL MAKE PIPELINE'''

'MUST STILL MAKE PIPELINE'

In [10]:
text_variables = ["title", "clean_body"]

In [11]:
## import elements from natural language toolkit
import nltk
#nltk.download('all') # uncomment after first run as admin check
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)
lmtzr = WordNetLemmatizer()

def get_tokens(line):
    '''
    Function to parse text features
    '''
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuations from each word
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stopwords
    words = [w for w in words if not w in stop_words]
    # lemmatizing the words, see https://en.wikipedia.org/wiki/Lemmatisation
    '''lemmatise or stem???'''
    words = [lmtzr.lemmatize(w) for w in words]
    # remove single letters
    words = [word for word in words if not len(word)==1]
    return (words)

In [40]:
import nltk

from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

class NLTKWordPunctTokenizer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(NLTKWordPunctTokenizer, self).__init__()
        self.stopwords = Param(self, "stopwords", "")
        self._setDefault(stopwords=set())
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, stopwords=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self

    def getStopwords(self):
        return self.getOrDefault(self.stopwords)

    def _transform(self, dataset):
        stopwords = self.getStopwords()

        def f(s):
            tokens = nltk.tokenize.wordpunct_tokenize(s)
            return [t for t in tokens if t.lower() not in stopwords]

        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [41]:
sentenceDataFrame = spark.createDataFrame([
  (0, "Hi I heard about Spark"),
  (0, "I wish Java could use case classes"),
  (1, "Logistic regression models are neat")
], ["label", "sentence"])

tokenizer = NLTKWordPunctTokenizer(
    inputCol="sentence", outputCol="words",  
    stopwords=set(nltk.corpus.stopwords.words('english')))

tokenizer.transform(sentenceDataFrame).show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|Hi I heard about ...|  [Hi, heard, Spark]|
|    0|I wish Java could...|[wish, Java, coul...|
|    1|Logistic regressi...|[Logistic, regres...|
+-----+--------------------+--------------------+



In [12]:
'''numerical_ensembler = VectorAssembler(inputCols=numerical_variables, outputCol="numerical_data")    

standardizer = StandardScaler(inputCol="numerical_data", outputCol="numerical_data_std")    

numerical_pipeline = Pipeline(stages=[numerical_ensembler, standardizer])

taxi_num = numerical_pipeline.fit(taxi).transform(taxi)

taxi_num.head(2)

processing_ensembler = VectorAssembler(inputCols=["categorical_encoding", "numerical_data"], 
                                         outputCol="features")  

processing_pipeline = Pipeline(stages=[categorical_pipeline, numerical_pipeline, processing_ensembler])

taxi_processed = processing_pipeline.fit(taxi).transform(taxi)

taxi_processed.head(2)

taxi_processed.head(2)[0].features.values'''

'numerical_ensembler = VectorAssembler(inputCols=numerical_variables, outputCol="numerical_data")    \n\nstandardizer = StandardScaler(inputCol="numerical_data", outputCol="numerical_data_std")    \n\nnumerical_pipeline = Pipeline(stages=[numerical_ensembler, standardizer])\n\ntaxi_num = numerical_pipeline.fit(taxi).transform(taxi)\n\ntaxi_num.head(2)\n\nprocessing_ensembler = VectorAssembler(inputCols=["categorical_encoding", "numerical_data"], \n                                         outputCol="features")  \n\nprocessing_pipeline = Pipeline(stages=[categorical_pipeline, numerical_pipeline, processing_ensembler])\n\ntaxi_processed = processing_pipeline.fit(taxi).transform(taxi)\n\ntaxi_processed.head(2)\n\ntaxi_processed.head(2)[0].features.values'

# Modeling

In [13]:
%%time

## tokenise and lemmatise using nltk function from import step
body_rdd = {}
for i in data_array:
    body_rdd[i] = datasets[i].select('clean_body').rdd.flatMap(lambda r: r).map(lambda line: (1, get_tokens(line)))

CPU times: user 8.55 ms, sys: 3.79 ms, total: 12.3 ms
Wall time: 470 ms


In [14]:
%%time

## look at first 10 terms from first post in english
print(body_rdd['english'].take(1)[0][1][:30])

['know', 'use', 'lay', 'use', 'lie', 'different', 'form', 'verb', 'always', 'getting', 'confused']
CPU times: user 21.7 ms, sys: 5.86 ms, total: 27.6 ms
Wall time: 5.54 s


'''RUN LATER'''

# get rid of stop words
doc_stopwords = {}
for i in data_array:
    doc_stopwords[i] = body_rdd[i].flatMap(lambda r: r[1]).map(lambda r: (r,1)).reduceByKey(lambda a,b: a+b)
    # here we assume that words that appear very frequently are stop words
    doc_stopwords[i] = doc_stopwords[i].filter(lambda a: a[1]>3000).map(lambda r: r[0]).collect()
    '''3000 threshold needs attention!'''
    # remove stopwords and single letters
    body_rdd[i] = body_rdd[i].map(lambda r: (r[0],[w for w in r[1] if not w in doc_stopwords[i] and not len(w)==1]))

In [15]:
%%time

from pyspark.ml.feature import CountVectorizer

## convert tokens into sparse vectors
body_df = {}
for i in data_array:
    body_df[i] = spark.createDataFrame(body_rdd[i], ["dummy","words"])
    body_df[i].cache()

CPU times: user 177 ms, sys: 25.4 ms, total: 202 ms
Wall time: 23.4 s


In [16]:
%%time
## check first english post vector
print(body_df['english'].take(1))

[Row(dummy=1, words=['know', 'use', 'lay', 'use', 'lie', 'different', 'form', 'verb', 'always', 'getting', 'confused'])]
CPU times: user 10.4 ms, sys: 6.71 ms, total: 17.1 ms
Wall time: 1min 56s


In [17]:
## garbage collector to speed up computation
import gc
collected = gc.collect()
print("Garbage collector: collected %d objects." % collected)

Garbage collector: collected 160 objects.


In [18]:
%%time
print(datetime.now().time())
'''BOTTLENECK TIME - JUST USE english FOR NOW'''
data_array = ['english']

cntvcr = CountVectorizer(inputCol="words", outputCol="features", minDF=2)

cntvcr_models = {}
word_feat_list = {}
for i in data_array:
    cntvcr_models[i] = cntvcr.fit(body_df[i])
    word_feat_list[i] = cntvcr_models[i].transform(body_df[i])
    word_feat_list[i].cache()

## show word vectors and feature counts for fora:
for i in data_array:
    print("\n------------------\n", i, "\n------------------\n")
    word_feat_list[i].show(10)

12:18:59.206030

------------------
 english 
------------------

+-----+--------------------+--------------------+
|dummy|               words|            features|
+-----+--------------------+--------------------+
|    1|[know, use, lay, ...|(44232,[6,15,32,5...|
|    1|[want, connect, t...|(44232,[2,3,6,21,...|
|    1|[possible, duplic...|(44232,[2,4,5,8,9...|
|    1|[say, correctly, ...|(44232,[8,217,265...|
|    1|[proven, proved, ...|(44232,[11,17,33,...|
|    1|[reading, audienc...|(44232,[0,1,3,4,5...|
|    1|[rule, thumb, pin...|(44232,[3,26,33,5...|
|    1|[two, task, calle...|(44232,[1,5,8,14,...|
|    1|[understand, term...|(44232,[6,9,23,32...|
|    1|[nothing, correct...|(44232,[1,2,10,13...|
+-----+--------------------+--------------------+
only showing top 10 rows

CPU times: user 20.5 ms, sys: 7.21 ms, total: 27.7 ms
Wall time: 6.82 s


In [19]:
%%time
print(datetime.now().time())

# converting pyspark.ml vectors to pyspark.mllib vectors

from pyspark.mllib.linalg import Vectors
def as_mllib_vector(v):
    return Vectors.sparse(v.size, v.indices, v.values)

features = {}
feature_vec_list = {}
for i in data_array:
    features[i] = word_feat_list[i].select("features")
    feature_vec_list[i] = features[i].rdd.map(lambda r: as_mllib_vector(r[0]))
    feature_vec_list[i].cache()

## print first pyspark.mllib vector across fora:
for i in data_array:
    print("\n------------------\n", i, "\n------------------\n")
    print(feature_vec_list[i].take(1))

12:19:06.034033

------------------
 english 
------------------

[SparseVector(44232, {6: 2.0, 15: 1.0, 32: 1.0, 52: 1.0, 55: 1.0, 91: 1.0, 221: 1.0, 452: 1.0, 945: 1.0, 2008: 1.0})]
CPU times: user 21 ms, sys: 7.92 ms, total: 28.9 ms
Wall time: 8.52 s


In [20]:
## look at the first 100 words of the vocabulary
for i in data_array:
    print("\n------------------\n", i, "\n------------------\n")
    print ("Vocabulary from CountVectorizerModel is:\n")
    print(cntvcr_models[i].vocabulary[:100])
    print("\n---\n")

    M = len(cntvcr_models[i].vocabulary)
    print("Number of terms M = ", M)


------------------
 english 
------------------

Vocabulary from CountVectorizerModel is:

['word', 'nt', 'sentence', 'would', 'one', 'like', 'use', 'example', 'say', 'used', 'correct', 'mean', 'english', 'question', 'something', 'know', 'phrase', 'meaning', 'could', 'time', 'way', 'two', 'people', 'term', 'following', 'think', 'also', 'want', 'someone', 'person', 'make', 'first', 'verb', 'thing', 'sound', 'find', 'looking', 'context', 'name', 'right', 'seems', 'difference', 'noun', 'usage', 'come', 'case', 'get', 'answer', 'need', 'dictionary', 'work', 'using', 'form', 'describe', 'see', 'different', 'good', 'even', 'however', 'understand', 'language', 'better', 'sure', 'many', 'may', 'help', 'please', 'said', 'go', 'another', 'book', 'part', 'found', 'expression', 'really', 'new', 'article', 'might', 'sense', 'ca', 'much', 'writing', 'wrong', 'speaker', 'thought', 'adjective', 'well', 'read', 'second', 'year', 'without', 'always', 'rule', 'day', 'feel', 'instead', 'take', 'whether',

### LDA Model

In [21]:
%%time
print(datetime.now().time())

## fit LDA models with k=10 topics
from pyspark.ml.clustering import LDA

lda = LDA(k=10, maxIter=5)

lda_model_list = {}
for i in data_array:
    lda_model_list[i] = lda.fit(word_feat_list[i])

12:19:14.912869
CPU times: user 19.1 ms, sys: 7.25 ms, total: 26.4 ms
Wall time: 12.9 s


%%time
print(datetime.now().time())
'''BOTTLENECK TIME - TAKES OVER 45 min JUST FOR ENGLISH'''

# calculate the perplexity and likelihood for each forum's model

loglik_list = {}
logper_list = {}
for i in data_array:
    loglik_list[i] = lda_model_list[i].logLikelihood(word_feat_list[i])
    logper_list[i] = lda_model_list[i].logPerplexity(word_feat_list[i])
    print("Finished", i, "at", datetime.now().time())

# low perplexity value indicates the model predicts the sample well

for i in data_array:
    print("The lower bound on the log likelihood of the " +'\033[1m'+ i +'\033[0m'+ " corpus is: " + str(loglik_list[i]))
    print("The upper bound on the perplexity of the " +'\033[1m'+ i +'\033[0m'+ " corpus is: " + str(logper_list[i]))
    print('\n')

In [22]:
%%time
print(datetime.now().time())

## first 5 words of the 10 topics from the LDA models per forum
topic_list = {}
for i in data_array:
    print("\n------------------\n", i, "\n------------------\n")
    topic_list[i] = lda_model_list[i].describeTopics(5)
    #print("The topics described by their top-weighted terms:\n")
    #topic_list[i].show(truncate=False)
    # show the results
    topic_j = topic_list[i].select("termIndices").rdd.map(lambda r: r[0]).collect()
    for j in topic_j:
        print(np.array(cntvcr_models[i].vocabulary)[j])
    print('\n')

12:19:27.816094

------------------
 english 
------------------

['word' 'sentence' 'example' 'use' 'would']
['sentence' 'mean' 'word' 'meaning' 'like']
['word' 'sentence' 'one' 'question' 'nt']
['nt' 'question' 'term' 'sentence' 'word']
['word' 'like' 'nt' 'something' 'example']
['word' 'one' 'mean' 'nt' 'someone']
['word' 'nt' 'one' 'say' 'time']
['word' 'meaning' 'sentence' 'example' 'one']
['word' 'nt' 'one' 'would' 'like']
['use' 'like' 'word' 'would' 'example']


CPU times: user 1.1 s, sys: 98.5 ms, total: 1.2 s
Wall time: 1.89 s


In [23]:
from pyspark.ml.regression import LinearRegression

In [24]:
lr = LinearRegression(maxIter=100,
                      regParam=0.3,
                      elasticNetParam=0.8,
                      #featuresCol="features",
                      labelCol="label",
                      predictionCol="prediction")

In [39]:
datasets['english'].select('y_ravi').count()

105475

In [34]:
word_feat_list['english'].count()

105475

In [36]:
word_feat_list['english'].withColumn("label", datasets['english']['y_ravi'])

AnalysisException: 'Resolved attribute(s) y_ravi#540 missing from dummy#610L,words#611,features#703 in operator !Project [dummy#610L, words#611, features#703, y_ravi#540 AS label#864].;;\n!Project [dummy#610L, words#611, features#703, y_ravi#540 AS label#864]\n+- Project [dummy#610L, words#611, UDF(words#611) AS features#703]\n   +- LogicalRDD [dummy#610L, words#611], false\n'

In [31]:
final_data = word_feat_list['english'].withColumn("label", datasets['english']['y_ravi'])

AnalysisException: 'Resolved attribute(s) y_ravi#540 missing from dummy#610L,words#611,features#703 in operator !Project [dummy#610L, words#611, features#703, y_ravi#540 AS label#829].;;\n!Project [dummy#610L, words#611, features#703, y_ravi#540 AS label#829]\n+- Project [dummy#610L, words#611, UDF(words#611) AS features#703]\n   +- LogicalRDD [dummy#610L, words#611], false\n'

In [ ]:
lr_model = lr.fit(finale)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(maxIter=100,
                      regParam=0.3,
                      elasticNetParam=0.8,
                      featuresCol="features",
                      labelCol="label",
                      predictionCol="prediction")
pipeline = Pipeline(stages=[processing_pipeline, lr])

In [ ]:
trained_pipeline = pipeline.fit(taxi)

In [ ]:
trained_pipeline

In [ ]:
trained_pipeline.transform(taxi).head()

In [ ]:
(trained_pipeline
 .transform(taxi)
 .select(
    categorical_variables + numerical_variables + ["prediction"]
 )
 .write
 .parquet("taxi_2014_prediction.parquet")
)

In [ ]:
taxi_predictions = spark.read.parquet("taxi_2014_prediction.parquet")

In [ ]:
taxi_predictions.head(2)

In [ ]:
taxi_predictions.select("prediction").describe().toPandas()

# Convert notebook to python file

In [ ]:
!jupyter nbconvert --to script 0-master-notebook.ipynb